<a href="https://colab.research.google.com/github/vinitakawale/Underwater-Image-Restoration/blob/main/UIRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/vinitakawale/Underwater-Image-Restoration.git

Cloning into 'Underwater-Image-Restoration'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 337 (delta 11), reused 180 (delta 10), pack-reused 156
Receiving objects: 100% (337/337), 51.13 MiB | 11.52 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [4]:
%cd Underwater-Image-Restoration/

/content/Underwater-Image-Restoration


In [5]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
import pytorch_ssim
import  time 
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.nn.modules.loss import _Loss 
from net.Ushape_Trans import *
#from dataset import prepare_data, Dataset
from net.utils import *
import cv2
import matplotlib.pyplot as plt
from utility import plots as plots, ptcolor as ptcolor, ptutils as ptutils, data as data
from loss.LAB import *
from loss.LCH import *
from torchvision.utils import save_image

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
my_path = "/saved_models" # Your path
gdrive_path = "/content/drive" + "/My Drive" + my_path
os.listdir(gdrive_path)

Mounted at /content/drive


['D', 'G']

In [8]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [9]:
def split(img):
    output=[]
    output.append(F.interpolate(img, scale_factor=0.125))
    output.append(F.interpolate(img, scale_factor=0.25))
    output.append(F.interpolate(img, scale_factor=0.5))
    output.append(img)
    return output

In [10]:
dtype = 'float32'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
torch.set_default_tensor_type(torch.FloatTensor)

In [12]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [13]:
# Initialize generator 
generator = Generator().cuda()
generator.load_state_dict(torch.load("saved_models/G/generator_795.pth"))

<All keys matched successfully>

In [14]:
generator.eval()

Generator(
  (linear_encoding): Linear(in_features=384, out_features=512, bias=True)
  (position_encoding): LearnedPositionalEncoding()
  (pe_dropout): Dropout(p=0.0, inplace=False)
  (transformer): TransformerModel(
    (net): IntermediateSequential(
      (0): Residual(
        (fn): PreNormDrop(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (fn): SelfAttention(
            (qkv): Linear(in_features=512, out_features=1536, bias=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=512, out_features=512, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
        )
      )
      (1): Residual(
        (fn): PreNorm(
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fn): FeedForward(
            (net): Sequential(
              (0): Linear(in_features=512, out_features=256, bias=True)
           

In [15]:
%cd /content/Underwater-Image-Restoration/

/content/Underwater-Image-Restoration


In [17]:
directory = "output"
parent_dir = "./test"
  
# Path
path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [18]:
path='./test/input/'#to change
path_list = os.listdir(path)
path_list.sort(key=lambda x:int(x.split('.')[0]))
i=1
for item in path_list:
    impath=path+item
    imgx= cv2.imread(path+item)
    imgx=cv2.resize(imgx,(256,256))
    imgx = cv2.cvtColor(imgx, cv2.COLOR_BGR2RGB)
    imgx = np.array(imgx).astype(dtype)

    imgx= torch.from_numpy(imgx)
    imgx=imgx.permute(2,0,1).unsqueeze(0)
    imgx=imgx/255.0
    #plt.imshow(imgx[0,:,:,:])
    #plt.show()
    imgx = Variable(imgx).cuda()
    #print(imgx.shape)
    output=generator(imgx)
    out=output[3].data
    save_image(out, "./test/output/"+item, nrow=5, normalize=True)
    i=i+1

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:3454: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


In [19]:
def compute_psnr(img1, img2):
   mse = np.mean( (img1/255. - img2/255.) ** 2 )
   if mse < 1.0e-10:
      return 100
   PIXEL_MAX = 1
   return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
def compute_mse(img1,img2):
    mse=np.mean( (img1/255. - img2/255.) ** 2 )
    return mse

In [20]:
path1='./test/GT/'
path2='./test/output/'
path_list = os.listdir(path1)
path_list.sort(key=lambda x:int(x.split('.')[0]))
PSNR=[]

for item in path_list:
    impath1=path1+item
    impath2=path2+item
    imgx= cv2.imread(impath1)
    imgx=cv2.resize(imgx,(256,256))
    imgy= cv2.imread(impath2)
    imgy=cv2.resize(imgy,(256,256))
    #print(imgx.shape)
    psnr1=compute_psnr(imgx[:,:,0],imgy[:,:,0])
    psnr2=compute_psnr(imgx[:,:,1],imgy[:,:,1])
    psnr3=compute_psnr(imgx[:,:,2],imgy[:,:,2])
    

    psnr=(psnr1+psnr2+psnr3)/3.0

    PSNR.append(psnr)

In [21]:
PSNR=np.array(PSNR)    
print(PSNR.mean())

24.748663941209653
